In [1]:
%config IPCompleter.use_jedi = False
import sys
import s3fs
import requests
import bs4
import pandas as pd
import numpy as np
from warcio.archiveiterator import ArchiveIterator

from distributed.diagnostics import progress
from distributed.client import Client
from distributed import worker_client
from dask import delayed
import dask.bag as db


client = Client('tcp://ec2-54-209-249-152.compute-1.amazonaws.com:8786')
client.restart()

/opt/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  """Entry point for launching an IPython kernel.


Client Scheduler: tcp://ec2-54-209-249-152.compute-1.amazonaws.com:8786 Dashboard: http://ec2-54-209-249-152.compute-1.amazonaws.com:8787,Cluster Workers: 2 Cores: 8 Memory: 18.93 GB


In [2]:
!pip install --upgrade requests

Requirement already up-to-date: requests in /opt/anaconda/lib/python2.7/site-packages
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /opt/anaconda/lib/python2.7/site-packages (from requests)
Requirement already up-to-date: idna<2.7,>=2.5 in /opt/anaconda/lib/python2.7/site-packages (from requests)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /opt/anaconda/lib/python2.7/site-packages (from requests)
Requirement already up-to-date: certifi>=2017.4.17 in /opt/anaconda/lib/python2.7/site-packages (from requests)
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
fs = s3fs.S3FileSystem(anon=True)

In [4]:
fs.ls('commoncrawl')

['commoncrawl/hive_analysis_$folder$',
 'commoncrawl/meanpath_$folder$',
 'commoncrawl/nutch_$folder$',
 'commoncrawl/robots.txt',
 'commoncrawl/2012',
 'commoncrawl/blekko',
 'commoncrawl/cc-index',
 'commoncrawl/contrib',
 'commoncrawl/crawl-001',
 'commoncrawl/crawl-002',
 'commoncrawl/crawl-analysis',
 'commoncrawl/crawl-data',
 'commoncrawl/finnougric-urls',
 'commoncrawl/hive_analysis',
 'commoncrawl/index2012',
 'commoncrawl/mapred-temp',
 'commoncrawl/meanpath',
 'commoncrawl/parse-output-test',
 'commoncrawl/parse-output',
 'commoncrawl/projects',
 'commoncrawl/stats-output',
 'commoncrawl/wikipedia']

In [6]:
fs.ls('commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc')[:10]

['commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00000-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00001-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00002-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00003-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00004-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00005-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00006-ip-10-60-113-184.ec2.internal.warc.gz',
 'commoncrawl/crawl-data/CC-MAIN-2

In [5]:
import time
import random

def add(n):
    time.sleep(random.random())
    return n + 1

def subtract(n):
    time.sleep(random.random())
    return n-1

results = []

for i in range(2000):
    result = delayed(add)(i)
    result = delayed(subtract)(result)
    results.append(result)
    


In [7]:
client.cancel(results)

In [2]:
def data_yielder(url):
    
    limit = 5000
    count = 0
    
    response = requests.get(url, stream=True)
        
    for i, record in enumerate(ArchiveIterator(response.raw)):

        if (record.rec_type == 'response' 
            and 'text/html' in record.http_headers.get_header('Content-Type', '')
            ):
            rec_headers = dict(record.rec_headers.headers)
            http_headers = dict(record.http_headers.headers)
            content = record.content_stream().read()

            yield {'rec_headers': rec_headers, 
                   'http_headers':http_headers,
                   'content': content}
            count += 1
            if count >= limit: break
            sys.stdout.write('\rCount: {}'.format(count))
            
def count_in_string(content, word):
    return content.lower().count(word.lower())    

def word_counter(url, word):
    count = 0
    for response_content in data_yielder(url):
        count += count_in_string(response_content['content'], word)
    return count
        
        

In [3]:
urls = [
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00000-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00001-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00002-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00003-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00004-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00005-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00006-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00007-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00008-ip-10-60-113-184.ec2.internal.warc.gz',
    'commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00009-ip-10-60-113-184.ec2.internal.warc.gz'
       ]
urls = ['http://s3.amazonaws.com/' + url for url in urls]

count = 0
for url in urls:
    count += delayed(word_counter)(url=url, word='big data')

In [4]:
%%time
result = client.compute(count)
result.result()

CPU times: user 100 ms, sys: 24 ms, total: 124 ms
Wall time: 1min 6s


In [6]:
result.result()

1329331

In [164]:
import bs4

def beautiful_soupify(data):
    import bs4
    html_content = data['content']
    soup = bs4.BeautifulSoup(html_content, 'html5lib')
    return soup.text

def count_word(html, word_phrase):
    return html.lower().count(word_phrase)

In [165]:
job = bag.map(beautiful_soupify)
job = job.map(count_word, word_phrase='big data')
job = job.sum()

In [166]:
job.compute()

174

In [144]:
html = bag.take(1)[0]['content']

In [146]:
html.count('meta')

8

In [162]:
client.restart()

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787,Cluster Workers: 4 Cores: 8 Memory: 12.00 GB
